# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 03: Training Pipeline</span>


<span style="font-width:bold; font-size: 1.4rem;">This notebook explains how to read from a feature group, create training dataset within the feature store, train a model and save it to model registry.</span>

## 🗒️ This notebook is divided into the following sections:

1. Fetch Feature Groups.
2. Define Transformation functions.
3. Create Feature Views.
4. Create Training Dataset with training, validation and test splits.
5. Train the model.
6. Register model in Hopsworks model registry.

![part2](../../images/02_training-dataset.png) 

## <span style='color:#ff5f27'> 📝 Imports

In [ ]:
!pip install xgboost --quiet

In [ ]:
import joblib
import os

import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

from sklearn.metrics import confusion_matrix
from xgboost import plot_importance
import xgboost as xgb
from sklearn.metrics import f1_score

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

## <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

## <span style="color:#ff5f27;">🪄 Retrieving Feature Groups</span>

In [ ]:
# Retrieve feature groups
applications_fg = fs.get_feature_group(
    name='applications',
    version=1,
)

bureau_balances_fg = fs.get_feature_group(
    name='bureau_balances',
    version=1,
)

bureaus_fg = fs.get_feature_group(
    name='bureaus',
    version=1,
)

previous_applications_fg = fs.get_feature_group(
    name='previous_applications',
    version=1,
)

pos_cash_balances_fg = fs.get_feature_group(
    name='pos_cash_balances',
    version=1,
)

installment_payments_fg = fs.get_feature_group(
    name='installment_payments',
    version=1,
)

credit_card_balances_fg = fs.get_feature_group(
    name='credit_card_balances',
    version=1,
)

previous_loan_counts_fg = fs.get_feature_group(
    name='previous_loan_counts',
    version=1,
)

---

## <span style="color:#ff5f27;">🕵🏻‍♂️ Feature Groups Investigation</span>

We can use `FeatureGroup.show()` method to select top n rows. 

Also we use method `FeatureGroup.read()` in order **to aggregate queries**, which are the output of next methods:

- `FeatureGroup.get_feture()` to get specific feature from our Feature Group.

- `FeatureGroup.select()` to get a few features from our Feature Group.

- `FeatureGroup.select_all()` to get all features from our Feature Group.

- `FeatureGroup.select_except()` to get all features except a few from our Feature Group.

- `FeatureGroup.filter()` in order to apply specific filter to the feature group.

In [ ]:
applications_fg

In [ ]:
applications_fg.select_all()

In [ ]:
applications_fg.read().head()

---

## <span style="color:#ff5f27;"> 💼 Feature Selection</span>

In [ ]:
# Build a query object 
query = bureaus_fg.select_except(['sk_id_curr','sk_id_bureau','datetime'])\
            .join(applications_fg.select_except(['sk_id_curr',
                                                 'datetime',
                                                 'flag_mobil',
                                                 *[f'flag_document_{num}'
                                                   for num
                                                   in [2,4,7,10,12,14,17,19,20,21]
                                                  ],
                                                 'amt_credit', 'weekday_appr_process_start',
                                                 'hour_appr_process_start']))\
            .join(bureau_balances_fg.select_except(['sk_id_bureau','months_balance']))\
            .join(previous_applications_fg.select_except(['sk_id_prev', 'sk_id_curr','datetime',
                                                          'name_contract_type', 'name_contract_status']))\
            .join(pos_cash_balances_fg.select_except(['sk_id_prev','sk_id_curr', 'months_balance',
                                                      'name_contract_status', 'sk_dpd', 'sk_dpd_def']))\
            .join(installment_payments_fg.select_except(['sk_id_prev', 'sk_id_curr', 'datetime']))\
            .join(credit_card_balances_fg.select_except(['sk_id_prev', 'sk_id_curr']))\
            .join(previous_loan_counts_fg.select_except('sk_id_curr'))

query_show5 = query.show(5)
query_show5

---

## <span style="color:#ff5f27;">🤖 Transformation functions</span>

Hopsworks Feature Store provides functionality to attach transformation functions to training datasets.

Hopsworks Feature Store also comes with built-in transformation functions such as `min_max_scaler`, `standard_scaler`, `robust_scaler` and `label_encoder`.

In [ ]:
# Retrieving the names of all available transformation functions
[t_func.name for t_func in fs.get_transformation_functions()]

We can retrieve transformation function we need .

To attach transformation function to training dataset provide transformation functions as dict, where key is feature name and value is online transformation function name.

Also training dataset must be created from the Query object. Once attached transformation function will be applied on whenever save, insert and get_serving_vector methods are called on training dataset object.

In [ ]:
# Extracting the names of categorical columns in the 'query_show5' DataFrame
cat_cols = query_show5.dtypes[query_show5.dtypes == 'object'].index

# Retrieving the Label Encoder transformation function from Featuretools
le = fs.get_transformation_function(name='label_encoder') 

# Creating a dictionary of transformation functions, where each categorical column is associated with the Label Encoder
transformation_functions = {
    col: le
    for col in cat_cols
}

---

## <span style="color:#ff5f27;"> ⚙️ Feature View Creation </span>

`Feature Views` stands between **Feature Groups** and **Training Dataset**. Сombining **Feature Groups** we can create **Feature Views** which store a metadata of our data. Having **Feature Views** we can create **Training Dataset**.

The Feature Views allows schema in form of a query with filters, define a model target feature/label and additional transformation functions.

In order to create Feature View we can use `FeatureStore.get_or_create_feature_view()` method.

We can specify next parameters:

- `name` - name of a feature group.

- `version` - version of a feature group.

- `labels`- out target variable.

- `transformation_functions` - functions to transform our features.

- `query` - query object with data.

In [ ]:
# Get or create the 'credit_scores' feature view
feature_view = fs.get_or_create_feature_view(
    name='credit_scores',
    version=1,
    labels=['target'],
    transformation_functions=transformation_functions,
    query=query,
)

---

## <span style="color:#ff5f27;"> 🏋️ Training Dataset Creation</span>

In Hopsworks training data is a query where the projection (set of features) is determined by the parent FeatureView with an optional snapshot on disk of the data returned by the query.

**Training Dataset  may contain splits such as:** 
* Training set - the subset of training data used to train a model.
* Validation set - the subset of training data used to evaluate hparams when training a model
* Test set - the holdout subset of training data used to evaluate a mode

To create training dataset you will use the `FeatureView.train_test_split()` method.

Here are some importand things:

- It will inherit the name of FeatureView.

- The feature store currently supports the following data formats for
training datasets: **tfrecord, csv, tsv, parquet, avro, orc**.

- You can choose necessary format using **data_format** parameter.

- **start_time** and **end_time** in order to filter dataset in specific time range.

- You can create **train, test** splits using `train_test_split()`. 

- You can create **train,validation, test** splits using `train_validation_test_splits()` methods.

- The only thing is that we should specify desired ratio of splits.

In [ ]:
X_train, X_test, y_train, y_test = feature_view.train_test_split(
    test_size=0.2,
)

---
## <span style="color:#ff5f27;"> 🧬 Modeling</span>

In [ ]:
# Creating an XGBoost Classifier model
xgboost_model = xgb.XGBClassifier()

# Training the XGBoost model on the training data
xgboost_model.fit(X_train, y_train)

---
## <span style="color:#ff5f27;"> 👨🏻‍⚖️ Model Evaluation</span>

In [ ]:
# Calculating the F1 score on the test set using the trained XGBoost model
score = f1_score(
    y_test, 
    xgboost_model.predict(X_test), 
    average="macro",
)

# Displaying the F1 score
print(f'⛳️ F1 score: {score}')

In [ ]:
# Calculating the confusion matrix on the test set using the trained XGBoost model
conf_matrix = confusion_matrix(
    y_test, 
    xgboost_model.predict(X_test),
)

# Creating a heatmap to visualize the confusion matrix
figure_cm = plt.figure(figsize=(10, 7))
figure_cm = sns.heatmap(
    conf_matrix, 
    annot=True, 
    annot_kws={"size": 14},
    fmt='.10g',
)

# Adding title to the heatmap
plt.title('Confusion Matrix', fontsize=17)

# Displaying the heatmap
plt.show()

In [ ]:
# Plotting feature importance using the plot_importance function from XGBoost
# Displaying the top 10 features based on their weight importance
# 'importance_type' is set to 'weight' to use the number of times a feature appears in a tree
figure_imp = plot_importance(
    xgboost_model, 
    max_num_features=10, 
    importance_type='weight',
)

# Displaying the feature importance plot
plt.show()

---
## <span style="color:#ff5f27;">🗄 Model Registry</span>

One of the features in Hopsworks is the model registry. This is where you can store different versions of models and compare their performance.

In [ ]:
# Retrieve the model registry
mr = project.get_model_registry()

### <span style="color:#ff5f27;">⚙️ Model Schema</span>

The model needs to be set up with a [Model Schema](https://docs.hopsworks.ai/machine-learning-api/latest/generated/model_schema/), which describes the inputs and outputs for a model.

A Model Schema can be automatically generated from training examples, as shown below.

In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

# Creating input and output schemas
input_schema = Schema(X_train.values)
output_schema = Schema(y_train)

# Creating a model schema
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

# Converting the model schema to a dictionary representation
schema_dict = model_schema.to_dict()

### <span style="color:#ff5f27;">💽 Save a model</span>

In [ ]:
# Creating a directory for the model artifacts if it doesn't exist
model_dir = "credit_scores_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

# Saving the trained XGBoost model as a joblib file in the model directory
joblib.dump(xgboost_model, model_dir + '/credit_scores_model.pkl')

# Saving the confusion matrix and feature importance plots as images in the model directory
figure_cm.figure.savefig(model_dir + '/confusion_matrix.png')
figure_imp.figure.savefig(model_dir + '/feature_importance.png')

In [ ]:
# Creating a Python model in the model registry named 'credit_scores_model'
model = mr.python.create_model(
    name="credit_scores_model",
    metrics={"f1_score": score}, 
    description="XGB for Credit Scores Project",
    input_example=X_train.sample(),
    model_schema=model_schema,
)

# Saving the model artifacts to the 'credit_scores_model' directory in the model registry
model.save(model_dir)

## <span style="color:#ff5f27;">⏭️ **Next:** Part 04: Batch Inference </span>

In the next notebook you will use your registered model to predict batch data.
